Notebook to run the glmm analysis per brain region and cell-type with papermill
originally coded to run in parallel out of a single notebook, but always have a probably passing a anndata object to ray or concurrent futures

so here running via papermill where notebook for each is run in parallel using threading library, have also previouls tried with ray and concurrent futures, all blow up papermill managing notebook blows up but individual notebooks keep running in parallel thru sucessful completion

In [1]:
!date

#### import libraries and set notebook variables

In [2]:
import os
import papermill as pm
import threading

import warnings
warnings.simplefilter('ignore')

/home/gibbsr/anaconda3/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [3]:
# naming
cohort = 'aging'

# directories
home_dir = f'/labshare/raph/notebooks/expression/adrd_neuro/{cohort}'

# base notebook to be run per tuple iteration
base_notebook = f'{home_dir}/glmm_diffexp.ipynb'

# output path for the generated notebooks
out_nb_dir = f'{home_dir}/pm_gend_nbs'

# setup parameters to iterate
brain_regions = ['Entorhinal cortex', 'Putamen', 'Subventricular zone', 
                 'Middle temporal gyrus']
cell_types = ['Oligodendrocyte-1', 'SPN D1', 'SPN D2', 'Oligodendrocyte-2', 
              'Astrocyte', 'ExN CUX2 LAMP5', 'InN ADARB2 VIP', 'ExN FEZF2', 
              'OPC', 'ExN RORB THEMIS', 'InN LHX6 PVALB', 'Radial Glia', 
              'Microglia', 'InN ADARB2 LAMP5', 'ExN CUX2 ADARB2', 
              'InN LHX6 SST', 'SPN D1-2', 'Endothelial', 'ExN RORB', 
              'ExN LAMP5', 'SPN D2-2', 'ExN THEMIS']
groupings = {'Brain_region': brain_regions, 'new_anno': cell_types}

testing = False

#### utility functions

In [4]:
def run_pm_notebook(base_notebook: str, out_notebook: str, params: dict) -> str:
    ret_val = f'notebook: {out_notebook}\nparams: {params}'
    pm.execute_notebook(input_path=base_notebook, output_path=out_notebook, 
                        parameters=params, progress_bar=False)
    return ret_val

#### check the notebook template

In [5]:
pm.inspect_notebook(base_notebook)

{'tissue': {'name': 'tissue',
  'inferred_type_name': 'None',
  'default': "''",
  'help': ''},
 'tissue_type': {'name': 'tissue_type',
  'inferred_type_name': 'None',
  'default': "''",
  'help': ''},
 'testing': {'name': 'testing',
  'inferred_type_name': 'None',
  'default': 'False',
  'help': ''}}

#### iterate over the list running the notebook per sample pool

In [ ]:
%%time
# make sure the notebook output dir exists
os.makedirs(out_nb_dir, exist_ok=True)

job_threads = []
for g_type, groups in groupings.items():
    for grouping in groups:
        param_dict = {'tissue': grouping, 'tissue_type': g_type, 'testing': testing}
        out_notebook = f'{out_nb_dir}/{grouping.replace(" ", "_")}.glmm_diffexp.ipynb'
        print(f'notebook: {out_notebook}\nparams: {param_dict}')
        this_thread = threading.Thread(target=run_pm_notebook, 
                                       args=(base_notebook, out_notebook, param_dict,))
        job_threads.append(this_thread)
        this_thread.start()
        
for job_thread in job_threads:
    job_thread.join()     

notebook: /labshare/raph/notebooks/expression/adrd_neuro/aging/pm_gend_nbs/Entorhinal_cortex.glmm_diffexp.ipynb
params: {'tissue': 'Entorhinal cortex', 'tissue_type': 'Brain_region', 'testing': False}
notebook: /labshare/raph/notebooks/expression/adrd_neuro/aging/pm_gend_nbs/Putamen.glmm_diffexp.ipynb
params: {'tissue': 'Putamen', 'tissue_type': 'Brain_region', 'testing': False}
notebook: /labshare/raph/notebooks/expression/adrd_neuro/aging/pm_gend_nbs/Subventricular_zone.glmm_diffexp.ipynb
params: {'tissue': 'Subventricular zone', 'tissue_type': 'Brain_region', 'testing': False}
notebook: /labshare/raph/notebooks/expression/adrd_neuro/aging/pm_gend_nbs/Middle_temporal_gyrus.glmm_diffexp.ipynb
params: {'tissue': 'Middle temporal gyrus', 'tissue_type': 'Brain_region', 'testing': False}
notebook: /labshare/raph/notebooks/expression/adrd_neuro/aging/pm_gend_nbs/Oligodendrocyte-1.glmm_diffexp.ipynb
params: {'tissue': 'Oligodendrocyte-1', 'tissue_type': 'new_anno', 'testing': False}
noteboo

2021-11-05 10:27:53.988689: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:54.352123: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:54.529413: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:55.194622: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:55.342447: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:55.538404: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-05 10:27:55.745986: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic libr

In [10]:
!date

Sun Nov  7 19:36:03 EST 2021
